In [1]:
import tensorflow
print(tensorflow.__version__)

C:\Users\Admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2.5.0


#Data Preprocessing

In [2]:
import cv2,os
data_path='I:/projects/cancer_detection_app'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'NORMAL': 0, 'PNEUMONIA': 1}
['NORMAL', 'PNEUMONIA']
[0, 1]


In [3]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

Exception: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Exception: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



#Recale and assign  catagorical lables

In [4]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [5]:
new_target.shape

(5216, 2)

#CNN Model

In [6]:
data.shape

(5216, 100, 100, 3)

In [7]:
data.shape[1:]

(100, 100, 3)

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 200)       5600      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 200)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0

#Splittiong data into traning and testing

In [10]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [11]:
train_data.shape

(4694, 100, 100, 3)

In [12]:
train_target.shape

(4694, 2)

In [13]:
#checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=50,validation_split=0.2)

Epoch 1/50
118/118 [==============================] - 140s 1s/step - loss: 0.5460 - accuracy: 0.7610 - val_loss: 0.1033 - val_accuracy: 0.9574
Epoch 2/50
118/118 [==============================] - 121s 1s/step - loss: 0.1449 - accuracy: 0.9464 - val_loss: 0.1114 - val_accuracy: 0.9627
Epoch 3/50
118/118 [==============================] - 121s 1s/step - loss: 0.1565 - accuracy: 0.9452 - val_loss: 0.0891 - val_accuracy: 0.9617
Epoch 4/50
118/118 [==============================] - 118s 1s/step - loss: 0.1001 - accuracy: 0.9571 - val_loss: 0.2096 - val_accuracy: 0.9244
Epoch 5/50
118/118 [==============================] - 119s 1s/step - loss: 0.0932 - accuracy: 0.9679 - val_loss: 0.0914 - val_accuracy: 0.9691
Epoch 6/50
118/118 [==============================] - 123s 1s/step - loss: 0.0758 - accuracy: 0.9740 - val_loss: 0.0943 - val_accuracy: 0.9702
Epoch 7/50
118/118 [==============================] - 119s 1s/step - loss: 0.0824 - accuracy: 0.9702 - val_loss: 0.0820 - val_accuracy: 0.9691

In [14]:
model.save('cancer_detection_model.h5')